In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

# 영화 이름을 가지고 감독, 주요 출연진, 예산, 흥행 수익, 영화의 장르, 간단한 시놉시스 등 영화에 대한 정보로 답장하는 체인을 만드세요.
# LLM은 항상 동일한 형식을 사용하여 응답해야 하며, 이를 위해서는 원하는 출력의 예시를 LLM에 제공해야 합니다.
# 예제를 제공하려면 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하세요.

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [

    {
        "movie_name": "Inception",
        "answer": """
        Here is what I know:
        Director: Christopher Nolan
        Main Cast: Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page
        Budget: $160 million
        Box Office: $829.9 million
        Genre: Science Fiction, Thriller
        Synopsis: A skilled thief, who enters the dreams of others to steal secrets, is tasked with planting an idea in someone's mind, a process known as inception.
        """
    },
    {
        "movie_name": "Parasite",
        "answer": """
        I know this:
        Director: Bong Joon-ho
        Main Cast: Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong
        Budget: $11 million
        Box Office: $263 million
        Genre: Drama, Thriller
        Synopsis: A poor family schemes to become employed by a wealthy family by infiltrating their household, leading to unforeseen consequences.
        """
    },
    {
        "movie_name": "The Godfather",
        "answer": """
        I know this:
        Director: Francis Ford Coppola
        Main Cast: Marlon Brando, Al Pacino, James Caan
        Budget: $6 million
        Box Office: $250-291 million
        Genre: Crime, Drama
        Synopsis: The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.
        """
    },

]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {movie_name}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {movie_name}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"movie_name": "3 idiots"})




        I know this:
        Director: Rajkumar Hirani
        Main Cast: Aamir Khan, R. Madhavan, Sharman Joshi
        Budget: ₹55 crore
        Box Office: ₹460 crore
        Genre: Comedy, Drama
        Synopsis: Two friends embark on a quest to find their long-lost college buddy, with the help of a free-spirited engineering student, and discover the meaning of true success.
        

AIMessageChunk(content='\n        I know this:\n        Director: Rajkumar Hirani\n        Main Cast: Aamir Khan, R. Madhavan, Sharman Joshi\n        Budget: ₹55 crore\n        Box Office: ₹460 crore\n        Genre: Comedy, Drama\n        Synopsis: Two friends embark on a quest to find their long-lost college buddy, with the help of a free-spirited engineering student, and discover the meaning of true success.\n        ')